In [1]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
src_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
if src_dir not in sys.path:
    sys.path.append(src_dir)
from modules.preprocess import *

In [2]:
model_df=pd.read_excel("../../data/250224 model_STD.xlsx")
maker_df=pd.read_excel("../../data/250224 maker_STD.xlsx")

In [3]:
maker_df

,makerID,makerSTDName,makerAlias,CLEAN_MAKER_NM
0,MA273101,Bomyeong Hitech,Bomyeong Hitech,BOMYEONGHITECH
1,MA1538,Nexstar,"NEX Star,Nexstar",NEXSTAR
2,MA298455,NAMYOUNG,NAMYOUNG,NAMYOUNG
3,MA298457,"DHE CO.,LTD","DHE CO.,LTD",DHECO.LTD
4,MA278365,SAFEX,SAFEX,SAFEX
...,...,...,...,...
3778,MA999,ISOTECH,ISOTECH,ISOTECH
3779,MA973,INTERGEN,INTERGEN,INTERGEN
3780,MA932,Iglika,Iglika,IGLIKA
3781,MA97,ALPHA AUTOMATION INC,"ALPHA,ALPHA AUTOMATION INC",ALPHAAUTOMATIONINC


In [4]:
explode_model_df = model_df[model_df["modelSTDName"] != "Unknown"]
explode_maker_df = maker_df[maker_df["makerSTDName"] != "Unknown"]

In [5]:
explode_model_df=alias_info_dataframe(explode_model_df)
explode_maker_df=alias_info_dataframe(explode_maker_df,"makerAlias")
explode_model_df["modelAlias"]=explode_model_df["modelAlias"].apply(clean_unknown_to_empty)
explode_model_df["modelAlias"]=explode_model_df["modelAlias"].apply(normalize_query)
explode_maker_df["makerAlias"]=explode_maker_df["makerAlias"].apply(clean_unknown_to_empty)
explode_maker_df["makerAlias"]=explode_maker_df["makerAlias"].apply(normalize_query)

In [6]:
# "ltd" 단독 행이 존재할 때 처리
match_idx = explode_maker_df[explode_maker_df["makerAlias"] == "ltd"].index
explode_maker_df.drop(index=match_idx, inplace=True)


In [7]:
explode_model_df.drop_duplicates(subset=["modelAlias"], inplace=True)
explode_model_df["modelAlias"] = explode_model_df["modelAlias"].replace("", np.nan)
explode_model_df.dropna(subset=["modelAlias"], inplace=True)
explode_maker_df.drop_duplicates(subset=["makerAlias"], inplace=True)
explode_maker_df["makerAlias"] = explode_maker_df["makerAlias"].replace("", np.nan)
explode_maker_df.dropna(subset=["makerAlias"], inplace=True)

In [8]:
alias_drop_explode_model_df = explode_model_df.drop(explode_model_df.index[[6986, 7045, 18033, 18346, 18402, 18433, 22199, 22201, 22724, 24027, 25286, 25289, 25290, 
                             27747, 27748, 376920, 378597, 378879, 381516, 381520,381536, 382778, 382806, 382956, 384236, 386470, 387375, 
                             389050, 390545, 390566,392985, 394117, 394633, 394716, 395541, 395749, 396224, 396556, 396675, 397774,397833, 
                             397869, 400237, 400340, 401669, 405003, 405671]], )

In [ ]:
from rapidfuzz import process, fuzz

def match_model_alias(user_input, model_df, alias_col="modelAlias", std_col="modelSTDName"):
    choices = model_df[alias_col].tolist()
    match, score, idx = process.extractOne(user_input, choices, scorer=fuzz.ratio)
    return {
        "user_input": user_input,
        "matched_alias": match,
        "matched_score": score,
        "matched_std_name": model_df.iloc[idx][std_col]
    }

def match_maker_alias(user_input, maker_df, alias_col="makerAlias", std_col="makerSTDName"):
    choices = maker_df[alias_col].tolist()
    match, score, idx = process.extractOne(user_input, choices, scorer=fuzz.token_sort_ratio)
    return {
        "user_input": user_input,
        "matched_alias": match,
        "matched_score": score,
        "matched_std_name": maker_df.iloc[idx][std_col]
    }

class StandardNameMatcher:
    def __init__(self, model_df=None, maker_df=None):
        self.model_df = model_df
        self.maker_df = maker_df

    def match_model(self, user_model_input):
        if self.model_df is None:
            raise ValueError("Model DataFrame is not provided.")
        return match_model_alias(user_model_input, self.model_df)

    def match_maker(self, user_maker_input):
        if self.maker_df is None:
            raise ValueError("Maker DataFrame is not provided.")
        return match_maker_alias(user_maker_input, self.maker_df)

    def match(self, user_model_input, user_maker_input):
        return {
            "model": self.match_model(user_model_input),
            "maker": self.match_maker(user_maker_input)
        }


In [ ]:
# tolist를 붙이는가 안붙이는가에 따라 index 변동있으므로 주의
process.extract("centura dpn radiance", alias_drop_explode_model_df["modelAlias"].tolist(), scorer=fuzz.ratio,score_cutoff=70,limit=100)

[('centura dpn radiance+', 97.5609756097561, 401266),
 ('centura dpn+radiance', 95.0, 386979),
 ('8" centura dpn radiance', 93.02325581395348, 390199),
 ('centura dpn radiance 8"', 93.02325581395348, 390200),
 ('centura dpn+radiance+', 92.6829268292683, 386955),
 ('centura rp radiance', 92.3076923076923, 18539),
 ('12" centura dpn radiance', 90.9090909090909, 385332),
 ('centura dpn radiance 12"', 90.9090909090909, 385334),
 ('centura acp radiance', 90.0, 18323),
 ('centura rtp radiance', 90.0, 18540),
 ('centura radiance', 88.88888888888889, 18531),
 ('8" centura dpn+radiance', 88.37209302325581, 400905),
 ('centura dpn+radiance 8"', 88.37209302325581, 400906),
 ('centura tpcc radiance', 87.8048780487805, 18542),
 ('centura acp radiance+', 87.8048780487805, 30594),
 ('centura radiance+', 86.48648648648648, 18328),
 ('centura acp dpn/radiance', 86.36363636363636, 18524),
 ('12" centura dpn+radiance', 86.36363636363636, 386962),
 ('centura dpn+radiance 12"', 86.36363636363636, 386963),


In [58]:
matcher = StandardNameMatcher(model_df=alias_drop_explode_model_df, maker_df=explode_maker_df)

# 모델만
print(matcher.match_model("centura dpn radiance"))

# 메이커만
print(matcher.match_maker("5300 hdp"))

# 둘 다
print(matcher.match("p5000 akt", "applied"))


{'user_input': 'centura dpn radiance', 'matched_alias': 'centura dpn radiance+', 'matched_score': 97.5609756097561, 'matched_std_name': 'Centura DPN Radiance+'}
{'user_input': '5300 hdp', 'matched_alias': 'ssm 5130', 'matched_score': 50.0, 'matched_std_name': 'SSM'}
{'model': {'user_input': 'p5000 akt', 'matched_alias': 'p5000 t', 'matched_score': 87.5, 'matched_std_name': 'P5000'}, 'maker': {'user_input': 'applied', 'matched_alias': 'applied', 'matched_score': 100.0, 'matched_std_name': 'AMAT'}}


In [ ]:
from tqdm import tqdm
mismatch_results = []

for i, test_model in tqdm(enumerate(explode_model_df["modelAlias"].tolist())):
    expected = explode_model_df["modelSTDName"].iloc[i]
    result = matcher.match_model(test_model)
    predicted = result["matched_std_name"]

    if expected != predicted:
        mismatch_results.append({
            "index": i,
            "input_alias": test_model,
            "expected_std_name": expected,
            "predicted_std_name": predicted,
            "score": result["matched_score"]
        })

# 결과를 DataFrame으로 보기
mismatch_df = pd.DataFrame(mismatch_results)

1459it [00:04, 353.57it/s]


KeyboardInterrupt: 

In [81]:
mismatch_df=pd.read_csv("../../data/mismatch_model.csv")
mismatch_df

,index,input_alias,expected_std_name,predicted_std_name,score
0,6986,9400se alliance (a4),TCP9400,TCP9400SE,100.0
1,7045,al endura,Endura,Endura 2,100.0
2,18033,ultima hdp cvd,0270-18073,Centura Ultima,100.0
3,18346,centura epi acp,Centura EPI,Centura ACP EPI,100.0
4,18402,epi centura acp,Centura EPI,Centura ACP EPI,100.0
5,18433,centura htf,Centura RTP,Centura,100.0
6,22199,centura ii dps metal,Centura2 DPS Metal,Centura DPS Metal,100.0
7,22201,centura ii dps metal etch,Centura2 DPS Metal,Centura DPS2 Metal,100.0
8,22724,870 s,SRD-870S,SRD-870,100.0
9,24027,5300 hdp,Centura 5300,Centura UltimaX,100.0


In [83]:
# modelID를 mismatch_df에 추가하는 코드
mismatch_df["expected_modelID"] = mismatch_df["index"].apply(lambda idx: explode_model_df.iloc[idx]["modelID"])
mismatch_df["predicted_modelID"] = mismatch_df["predicted_std_name"].apply(lambda stdname: explode_model_df[explode_model_df["modelSTDName"]==stdname]["modelID"].iloc[0])

In [86]:
mismatch_df

,index,input_alias,expected_std_name,predicted_std_name,score,expected_modelID,predicted_modelID
0,6986,9400se alliance (a4),TCP9400,TCP9400SE,100.0,MO2530,MO2531
1,7045,al endura,Endura,Endura 2,100.0,MO849,MO869
2,18033,ultima hdp cvd,0270-18073,Centura Ultima,100.0,MO453814,MO474
3,18346,centura epi acp,Centura EPI,Centura ACP EPI,100.0,MO4664,MO285267
4,18402,epi centura acp,Centura EPI,Centura ACP EPI,100.0,MO4664,MO285267
5,18433,centura htf,Centura RTP,Centura,100.0,MO382,MO383
6,22199,centura ii dps metal,Centura2 DPS Metal,Centura DPS Metal,100.0,MO275024,MO274713
7,22201,centura ii dps metal etch,Centura2 DPS Metal,Centura DPS2 Metal,100.0,MO275024,MO275022
8,22724,870 s,SRD-870S,SRD-870,100.0,MO9791,MO9790
9,24027,5300 hdp,Centura 5300,Centura UltimaX,100.0,MO126176,MO477


In [85]:
mismatch_df.to_csv("../../data/mismatch_alias.csv", index=False)

centura

입력하는 정책이 있어서 그렇게 진행한것인지.
따로따로 입력하자는 정책이있어서 그렇게 한것인지.
모델을 생성할때 나누는 기준은 무엇이었는지
재정리를 한다고 할지라도 기준을 세워놓고 정리하지 않으면 똑같은일이 반복되기때문에 기준을 성립해야한다.

기계적인 방식을 진행한다면 configure를 적는기준을 세워야한다.

configuration을 분류할때 어떻게 분류를 하게 되는것인지?

pair paeasing 로 확인해볼것
cross attention 다른 모달리티


병렬로 연결되어 있는 연결들. 확인해볼것.


입력 - > p5000 -> 100개
정리를 해야한다.
relevance feedback
1. 사양에 따라 configuration에 표현이 된다.


In [89]:
indexnum=18346
explode_model_df.iloc[indexnum]

modelID                  MO4664
modelSTDName        Centura EPI
MakerID                   MA110
category                    CVD
modelAlias      centura epi acp
Name: 18925, dtype: object

In [91]:
explode_model_df[explode_model_df["modelID"]=="MO4664"]

,modelID,modelSTDName,MakerID,category,modelAlias
18917,MO4664,Centura EPI,MA110,CVD,.centura acp
18918,MO4664,Centura EPI,MA110,CVD,1 wcvd ch )
18919,MO4664,Centura EPI,MA110,CVD,5200 centura epi
18920,MO4664,Centura EPI,MA110,CVD,5200 centura ht epi
18921,MO4664,Centura EPI,MA110,CVD,amat centura epi
...,...,...,...,...,...
18989,MO4664,Centura EPI,MA110,CVD,epi centura acp sige
18990,MO4664,Centura EPI,MA110,CVD,epi centura5200
18991,MO4664,Centura EPI,MA110,CVD,epi centuras
18993,MO4664,Centura EPI,MA110,CVD,ht centura


In [90]:
explode_maker_df[explode_maker_df["makerID"]==explode_model_df.iloc[indexnum]["MakerID"]]

,makerID,makerSTDName,CLEAN_MAKER_NM,makerAlias
339,MA110,AMAT,AMAT,0190 00398
340,MA110,AMAT,AMAT,0190 01292r
341,MA110,AMAT,AMAT,akt
342,MA110,AMAT,AMAT,ama
343,MA110,AMAT,AMAT,amaat
...,...,...,...,...
447,MA110,AMAT,AMAT,ｱﾌﾟﾗｲﾄﾞﾏﾃﾘｱﾙ
448,MA110,AMAT,AMAT,アプライドマテリアルズ
449,MA110,AMAT,AMAT,ｱﾌﾟﾗｲﾄﾞﾏﾃﾘｱﾙｽﾞｼﾞｬﾊﾟﾝ
450,MA110,AMAT,AMAT,アプライドマテリアルズジャパン株式会社


In [77]:
explode_model_df[explode_model_df["modelSTDName"]=="Archer 200 AIM"]

,modelID,modelSTDName,MakerID,category,modelAlias
31767,MO278747,Archer 200 AIM,MA261097,Metrology,archer 200 aim


In [16]:
explode_maker_df[explode_maker_df["makerID"]=="MA110"]

,makerID,makerSTDName,CLEAN_MAKER_NM,makerAlias
339,MA110,AMAT,AMAT,0190 00398
340,MA110,AMAT,AMAT,0190 01292r
341,MA110,AMAT,AMAT,akt
342,MA110,AMAT,AMAT,ama
343,MA110,AMAT,AMAT,amaat
...,...,...,...,...
447,MA110,AMAT,AMAT,ｱﾌﾟﾗｲﾄﾞﾏﾃﾘｱﾙ
448,MA110,AMAT,AMAT,アプライドマテリアルズ
449,MA110,AMAT,AMAT,ｱﾌﾟﾗｲﾄﾞﾏﾃﾘｱﾙｽﾞｼﾞｬﾊﾟﾝ
450,MA110,AMAT,AMAT,アプライドマテリアルズジャパン株式会社


In [73]:
mismatch_df.to_csv("../../data/mismatch_model.csv", index=False)

In [ ]:
from tqdm import tqdm
mismatch_results = []

for i, test_maker in tqdm(enumerate(explode_maker_df["makerAlias"].tolist())):
    expected = explode_maker_df["makerAlias"].iloc[i]
    result = matcher.match_model(test_maker)
    predicted = result["matched_std_name"]

    if expected != predicted:
        mismatch_results.append({
            "index": i,
            "input_alias": test_maker,
            "expected_std_name": expected,
            "predicted_std_name": predicted,
            "score": result["matched_score"]
        })

# 결과를 DataFrame으로 보기
mismatch_df = pd.DataFrame(mismatch_results)
